Warp Radolan data to GTIFF 

In [ ]:
#Import Modules
import wradlib as wrl
import numpy as np
import warnings
warnings.filterwarnings('ignore')

#Working dir
wdir = "/home/elisabeth/Dokumente/master/radolan_data/raw/"
file = "raa01-sf_10000-1701012350-dwd---bin"
#Read the original data: data = 2d Array(row,col) and metadata (productType, datetime,nodataflag)
#Origin in lower left corner
data_raw, meta  = wrl.io.read_radolan_composite(wdir + file)

#Round data to 2 decimal places to save memory
data_raw_s = np.around(data_raw, decimals = 2, out = None)

# Create Spatail Reference (Radolan projection)
proj_osr = wrl.georef.create_osr("dwd-radolan")

#Load EPSG 3857 
epsg_3857 = wrl.georef.epsg_to_osr(3857)

# Get projected RADOLAN coordinates for corner definition
#origin in lower left corner
xy_raw = wrl.georef.get_radolan_grid(900, 900)

#flip the data and the coords up-down (from lower left to upper left corner)
data, xy = wrl.georef.set_raster_origin(data_raw, xy_raw, 'upper')

#Create raster dataset: Geotiff with radolan projection
ds = wrl.georef.create_raster_dataset(data, xy, projection=proj_osr)

#Reproject dataset to EPSG 3857
ds2 = wrl.georef.reproject_raster_dataset(ds,projection_target=epsg_3857, spacing=1000)
#Bug appears it will fixed in future https://groups.google.com/forum/#!topic/wradlib-users/-dvRhDCjgV0
#This line is needed until fixed
ds2.SetProjection(epsg_3857.ExportToWkt())

#write raster dataset
filename = file[15:25] + ".tif"
wrl.io.write_raster_dataset(wdir + filename, ds2, "GTiff")
#

In [ ]:
#Import Modules
import wradlib as wrl
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import os 

#Working dir paths must be clean otherwise you have to make list comprehension that only bin files in files list
outdir ="/media/elisabeth/Seagate 1TB/radolan_downloads/"
wdir = "/home/elisabeth/Dokumente/master/radolan_data/raw/radolan_downloads/"
out = "out"
#Change to Workingdir
os.chdir(outdir)

if not os.path.exists(out):
    os.mkdir(out)

os.chdir(wdir)
paths = os.listdir()

# Create Spatail Reference (Radolan projection)
proj_osr = wrl.georef.create_osr("dwd-radolan")

#Load EPSG 3857 
epsg_3857 = wrl.georef.epsg_to_osr(3857)

# Get projected RADOLAN coordinates for corner definition
#origin in lower left corner
xy_raw = wrl.georef.get_radolan_grid(900, 900)


for path in paths:
    os.chdir(path)
    print("Workingdir: ", os.getcwd())
    files = os.listdir()
    for file in files:
        print("Load: ",file ," data, georeferencing, reprojection and translate to Gtif...")
        data_raw, meta  = wrl.io.read_radolan_composite(os.getcwd() + "/"+ file)
        #flip the data and the coords up-down (from lower left to upper left corner)
        data, xy = wrl.georef.set_raster_origin(data_raw, xy_raw, 'upper')
        #Create raster dataset: Geotiff with radolan projection
        ds = wrl.georef.create_raster_dataset(data, xy, projection=proj_osr)
        #Reproject dataset to EPSG 3857
        ds2 = wrl.georef.reproject_raster_dataset(ds,projection_target=epsg_3857, spacing=1000)
        #Bug appears it will fixed in future https://groups.google.com/forum/#!topic/wradlib-users/-dvRhDCjgV0
        #This line is needed until fixed
        ds2.SetProjection(epsg_3857.ExportToWkt())
        #write raster dataset
        filename = file[15:25] + ".tif"
        wrl.io.write_raster_dataset(outdir + out + "/" + filename, ds2, "GTiff")
        print("Save", filename, "to", os.getcwd())
    os.chdir("../")

        

In [ ]:
#For Files located in single folder
#Import Modules
import wradlib as wrl
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import os 
from osgeo import osr


#Working dir paths must be clean otherwise you have to make list comprehension that only bin files in files list
wdir = "/home/elisabeth/Dokumente/master/radolan_data/out/bin/2009-2019/"
out = "out"
#Change to Workingdir
os.chdir(wdir)

if not os.path.exists(out):
    os.mkdir(out)


#create the necessary Spatial Reference Objects for the RADOLAN-projection 
proj_stereo = wrl.georef.create_osr("dwd-radolan")
proj_wgs = osr.SpatialReference()
proj_wgs.ImportFromEPSG(4326)
radolan_grid_xy = wrl.georef.get_radolan_grid(900,900)

# create Gauss Krueger zone 3 projection osr object
proj_gk3 = osr.SpatialReference()
proj_gk3.ImportFromEPSG(31467)

# transform radolan polar stereographic projection to wgs84 and then to gk3
radolan_grid_ll = wrl.georef.reproject(radolan_grid_xy,
                                       projection_source=proj_stereo,
                                       projection_target=proj_wgs)
radolan_grid_gk = wrl.georef.reproject(radolan_grid_ll,
                                       projection_source=proj_wgs,
                                       projection_target=proj_gk3)


#Get list of files inside the folder
files = os.listdir()
print(len(files)," files inside the folder")

for file in files[0:25]:
    print("Load: ",file ," data, georeferencing, reprojection and translate to Gtif...")
    data_raw, meta  = wrl.io.read_radolan_composite(file)
    #flip the data and the coords up-down (from lower left to upper left corner)
    data, xy = wrl.georef.set_raster_origin(data_raw, radolan_grid_xy, 'upper')
    #Create raster dataset: Geotiff with radolan projection
    ds = wrl.georef.create_raster_dataset(data, xy, projection=proj_stereo)
    #Reproject dataset to GK3
    ds2 = wrl.georef.reproject_raster_dataset(ds,projection_target=proj_gk3, spacing=1000)
    #Bug appears it will fixed in future https://groups.google.com/forum/#!topic/wradlib-users/-dvRhDCjgV0
    #This line is needed until fixed
    #ds2.SetProjection(epsg_3857.ExportToWkt())
    #write raster dataset
    filename = file[15:25] + ".tif"
    wrl.io.write_raster_dataset(wdir + out + "/" + filename, ds2, "GTiff")
    print("Save", filename, "to", os.getcwd())

